#  프롬프트 엔지니어링 - 효과적인 프롬프트 템플릿 설계

### **학습 목표:**  효과적인 프롬프트 템플릿의 기본 구조와 설계 원칙을 이해한다

### 필수 패키지 설치
```bash
uv add langchain-openai langchain-core python-dotenv defusedxml
```

### 환경 변수 설정
프로젝트 루트에 `.env` 파일을 생성하고 다음과 같이 OpenAI API 키를 설정합니다:
```
OPENAI_API_KEY=your_api_key_here
```

---

### 추가 필기
- 프롬프트 모범 사례 : https://platform.claude.com/docs/ko/build-with-claude/prompt-engineering/claude-prompting-best-practices
	- 구체적인 질문, 
- 교안
	- class 보다는 from_template 메소드를 사용하는 것을 더욱 권장한다.
		- class를 사용하는 경우 따로 변수 처리를 해야하므로 함수 방식을 더욱 권장한다.

# 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    temperature=0.3,
    top_p=0.9,
)

---
# **프롬프트 유형**

- 종류: 질문형, 지시형, 대화형, 조건부, 예시 기반 등 
- 이러한 프롬프트 유형들은 상황에 따라 조합하여 사용 가능
- 목적에 맞는 적절한 유형을 선택하는 것이 중요

`(1) 질문형 프롬프트 (Question Prompts)`
   - 정보 추출에 효과적
   - 구체적인 답변 유도 가능

In [ ]:
from langchain_core.prompts import PromptTemplate

# 단순 질문형
question_prompt = PromptTemplate(
    template="다음 주제에 대해 무엇을 알고 있나요?: {topic}",
    input_variables=["topic"]
)

# LCEL chain 구성
chain = question_prompt | llm

# 질문
topic = "양자 컴퓨팅"
output = chain.invoke({"topic": topic})
pprint(output.content)

In [ ]:
### from_template 메소드를 사용한 방법

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pprint import pprint

# from_template을 사용한 PromptTemplate 생성
question_template = "다음 주제에 대해 무엇을 알고 있나요?: {topic}"
question_prompt = PromptTemplate.from_template(question_template)

# LCEL chain 구성
chain = question_prompt | llm

# 질문 실행
topic = "양자 컴퓨팅"
output = chain.invoke({"topic": topic})
pprint(output.content)

**[참고]** `from_template` 방식의 장점:
- 코드가 더 간결해진다
- 입력 변수를 수동으로 지정할 필요가 없다
- 템플릿에서 사용된 변수를 자동으로 추출한다

In [ ]:
# 분석 질문형 
analysis_prompt = PromptTemplate(
    template="""다음 텍스트에서 주요 논점은 무엇인가요? 세 가지로 설명해주세요.

    [텍스트]
    {text}

    [답변]
    """,
    input_variables=["text"]
)

# LCEL
chain = analysis_prompt | llm

# 질문
text = """양자 컴퓨팅은 양자역학의 원리를 이용하여 정보를 처리하는 컴퓨팅 기술이다. 
양자 컴퓨팅은 전통적인 컴퓨팅과는 다르게 양자역학의 원리를 이용하여 정보를 처리한다. 
양자 컴퓨팅은 양자역학의 원리를 이용하여 정보를 처리하는 컴퓨팅 기술이다.
"""

output = chain.invoke({"text": text})
pprint(output.content)

### **[실습 1]**

- 앞의 예제를 from_template 메소드를 사용하는 방식으로 구현하세요.

In [ ]:
# 여기에 코드를 작성하세요.

`(2) 지시형 프롬프트 (Instruction Prompts)`
   - 명확한 작업 수행 지시
   - 단계별 처리 가능

In [ ]:
from langchain_core.output_parsers import StrOutputParser
# 작업 지시형
task_prompt = PromptTemplate(
    template="다음 텍스트를 한국어로 번역하세요:\n\n[텍스트]\n{text}",
    input_variables=["text"]
)

# LCEL
chain = task_prompt | llm | StrOutputParser()

# 질문
text = "Quantum computing is a computing technology that uses the principles of quantum mechanics to process information."
output = chain.invoke({"text": text})
pprint(output)

### 추가 필기
- 아래 작업 순서를 넣어주면 머신이 해당 순서대로 진행된다.
- nano 모델이어도 원하는 순서대로 진행이 된다.

In [ ]:
# 단계별 지시형
step_prompt = PromptTemplate(
    template="""다음 텍스트에 대해서 작업을 순서대로 수행하세요:

    [텍스트]
    {text}

    [작업 순서]
    1. 텍스트를 1문장으로 요약
    2. 핵심 키워드 3개 추출
    3. 감정 분석 수행(긍정/부정/중립)

    [작업 결과]
    """,
    input_variables=["text"]
)

# LCEL
chain = step_prompt | llm | StrOutputParser()

# 질문
text = """
양자 컴퓨팅은 양자역학의 원리를 바탕으로 데이터를 처리하는 새로운 형태의 계산 방식이다. 
기존의 고전적 컴퓨터는 0과 1로 이루어진 이진법(bit)을 사용하여 데이터를 처리하지만, 
양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용하여 훨씬 더 복잡하고 빠른 계산을 수행할 수 있다. 

큐비트는 동시에 0과 1의 상태를 가질 수 있는 양자 중첩(superposition) 상태를 활용하며, 
이를 통해 병렬 계산과 같은 고급 기능이 가능하다.
""" 

output = chain.invoke({"text": text})
pprint(output)

### **[실습 2]**

- 두 개의 문장을 입력받아서, 두 문장의 맥락이 일치하는지 여부를 비교 분석하는 체인을 구성하세요.
- PromptTemplate를 사용하고, 두 문장을 입력받을 수 있도록 합니다. 
- 단계별 지시형 프롬프트로 작성합니다. 

In [ ]:
# 여기에 코드를 작성하세요.

### [실습2 강사님 코드]

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의
context_prompt = PromptTemplate(
    template="""다음 두 문장의 맥락 일치성을 분석해주세요:

    [문장 1]
    {sentence1}

    [문장 2]
    {sentence2}

    [분석 단계]
    1. 각 문장의 주요 주제 파악
    2. 핵심 키워드 비교
    3. 문맥적 연관성 분석
    4. 일치성 점수 산출 (0-100%)

    [분석 결과 형식]
    주제 분석:
    키워드 비교:
    맥락 분석:
    일치성 점수:
    최종 판단: (일치/불일치)
    """,
    input_variables=["sentence1", "sentence2"]
)

# LCEL chain 구성
chain = context_prompt | llm

# 테스트 문장
sentence1 = "인공지능은 데이터를 기반으로 학습합니다."
sentence2 = "머신러닝 알고리즘은 데이터셋을 통해 패턴을 학습합니다."

# 질문 실행
output = chain.invoke({"sentence1": sentence1, "sentence2": sentence2})
pprint(output.content)

`(3) 대화형 프롬프트 (Conversational Prompts)`
   - 자연스러운 상호작용
   - 문맥 유지 가능

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatPromptTemplate 객체를 직접 생성 (메시지 리스트, 입력 변수 리스트)
chat_prompt = ChatPromptTemplate(
    messages=[
        ("system", "당신은 친절한 고객 서비스 담당자입니다."),
        ("human", "{customer_message}")
    ],
    input_variables=["customer_message"]
)

# LCEL chain 구성
chain = chat_prompt | llm | StrOutputParser()

# 질문 실행
customer_message = "안녕하세요. 제품 배송 문제로 연락드렸어요."
output = chain.invoke({"customer_message": customer_message})
pprint(output)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# from_messages 메소드를 사용한 ChatPromptTemplate 생성
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 고객 서비스 담당자입니다."),
    ("human", "{customer_message}"),
])

# LCEL
chain = chat_prompt | llm | StrOutputParser()

# 질문
customer_message = "안녕하세요. 제품 배송 문제로 연락드렸어요."
output = chain.invoke({"customer_message": customer_message})
pprint(output)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

# 메시지 템플릿 사용
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "당신은 친절한 고객 서비스 담당자입니다."
    ),
    HumanMessagePromptTemplate.from_template(
        "{customer_message}"
    )
])

# LCEL
chain = chat_prompt | llm | StrOutputParser()

# 질문
customer_message = "안녕하세요. 제품 배송 문제로 연락드렸어요."
output = chain.invoke({"customer_message": customer_message})
pprint(output)


### **[실습 3]**

- 상품 리뷰를 분석하는 AI 체인을 대화형 프롬프트로 구성합니다. 
- 메시지 템플릿을 사용하여 메시지 역할을 구분합니다. 
- role, message 구분해서 처리하는 방향으로 실습을 변경해도 된다.

In [ ]:
# 예시 리뷰
reviews = [
    "이 블루투스 이어폰 정말 만족스러워요! 음질도 좋고 배터리도 오래가요. 다만 케이스가 좀 큰 감이 있네요.",
    "배송은 빨랐는데 제품 품질이 기대에 못 미쳐요. 터치감이 둔하고 연결이 자주 끊깁니다. 가성비 생각하면 그냥 쓸만하네요.",
    "가격대비 괜찮은 것 같아요. 디자인도 깔끔하고 기본적인 기능은 다 갖췄네요. 추천합니다!"
]

In [ ]:
# 여기에 코드를 작성하세요.


# 각 리뷰 분석 실행
for idx, review in enumerate(reviews, 1):
    print(f"\n=== 리뷰 {idx} 분석 결과 ===")
    result = review_chain.invoke({"review": review})
    print(result)

### [실습 3 강사님 코드]

In [ ]:
# 리뷰 분석 템플릿 생성
review_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "당신은 상품 리뷰 분석 전문가입니다. 리뷰의 감정과 주요 포인트를 분석합니다."
        ),
    HumanMessagePromptTemplate.from_template(
        "다음 리뷰를 분석해주세요: {review}"
        )
])

# LCEL
review_chain = review_prompt | llm | StrOutputParser()

# 각 리뷰 분석 실행
for idx, review in enumerate(reviews, 1):
    print(f"\n=== 리뷰 {idx} 분석 결과 ===")
    result = review_chain.invoke({"review": review})
    print(result)

`(4) 예시 기반 프롬프트 (Few-Shot Prompts)`
   - 원하는 출력 형식 명확화
   - 모델의 이해도 향상

In [ ]:
few_shot_prompt = PromptTemplate(
    template="""다음은 텍스트를 요약하는 예시입니다:

원문: {example_input}
요약: {example_output}

이제 다음 텍스트를 같은 방식으로 50자 이내로 요약해주세요:
원문: {input_text}
요약:
""",
    input_variables=["example_input", "example_output", "input_text"]
)

# LCEL
chain = few_shot_prompt | llm | StrOutputParser()

# 예시 텍스트 
example_input = """인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 
컴퓨터 프로그램으로 실현한 기술이다. 인공지능은 딥러닝, 기계학습 등 다양한 기술을 포함하며, 
최근에는 자율주행, 의료진단, 언어번역 등 다양한 분야에서 활용되고 있다."""

example_output = "인공지능: 인간의 학습능력, 추론능력, 지각능력 등을 컴퓨터 프로그램으로 실현한 기술"

# 입력 텍스트 
input_text = """양자 컴퓨팅은 양자역학의 원리를 활용하여 정보를 처리하는 혁신적인 컴퓨팅 기술이다. 
기존의 디지털 컴퓨터가 0과 1의 이진법을 사용하는 것과 달리, 양자 컴퓨터는 중첩 상태를 활용하여 
동시에 여러 계산을 수행할 수 있다. 이러한 특성으로 인해 특정 문제에서는 기존 컴퓨터보다 
월등히 빠른 처리 속도를 보여준다. 현재는 아직 초기 단계지만, 암호화, 신약 개발, 기후 모델링 등 
다양한 분야에서 혁신적인 발전이 기대된다."""

# 체인 실행
output = chain.invoke({
    "example_input": example_input, 
    "example_output": example_output, 
    "input_text": input_text
})
pprint(output)

### **[실습 4]**

- [실습 3]의 상품 리뷰 분석 시스템의 입출력 형식을 예시로 추가해서 프롬프트를 작성합니다.
- 체인을 실해하여 테스트합니다. 

In [ ]:
# 여기에 코드를 작성하세요.


`(5) 조건부 프롬프트 (Conditional Prompts)`
   - 상황별 다른 처리
   - 유연한 응답 생성

In [ ]:
# 조건부 프롬프트 템플릿 정의 (입력 텍스트에 따라 작업 유형을 지정)
conditional_prompt = PromptTemplate(
    template="""입력 텍스트: {text}

주어진 텍스트가 질문인 경우: 명확한 답변을 제공
주어진 텍스트가 진술문인 경우: 진술문의 사실 여부를 검증
주어진 텍스트가 요청사항인 경우: 수행 방법을 단계별로 설명

응답은 다음 형식을 따라주세요:
유형: [질문/진술문/요청사항]
내용: [상세 응답]""",
    input_variables=["text"]
)

# LCEL 체인 구성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
chain = conditional_prompt | llm | StrOutputParser()

In [ ]:
# 질문형 테스트
response = chain.invoke({
    "text": "인공지능은 인간의 일자리를 모두 대체하게 될까요?",
})
print(response)

In [ ]:
# 진술문 테스트
response = chain.invoke({
    "text": "양자컴퓨터는 현재 모든 암호화 시스템을 무력화할 수 있다."
})
print(response)

In [ ]:
# 요청사항 테스트
response = chain.invoke({
        "text": "파이썬으로 간단한 웹 스크래퍼를 만들고 싶습니다."
})
print(response)

--- 

# **프롬프트 엔지니어링 개념**

1. **개념**:
    - AI 모델에게 효과적인 지시를 제공하여 원하는 결과를 얻어내는 기술
    - 입력(프롬프트)을 최적화하여 출력의 품질을 향상하는 방법 

2. **원칙**:

    1. **명확성(Clarity)**
        - 모호하지 않은 명확한 지시사항 제공
        - 구체적인 요구사항과 제약조건 명시
        - 예시: "5개의 짧은 문장으로 요약해주세요" vs "요약해주세요"

    2. **맥락성(Context)**
        - 관련 배경 정보 제공
        - 목적과 의도 명시
        - 대상 독자나 사용 환경 설명

    3. **구조화(Structure)**
        - 체계적인 형식 사용
        - 단계별 지시사항 제공
        - 원하는 출력 형식 명시

---
### 추가 필기

- 프롬프트를 작성할 때 
 1. 예시 텍스트와 비슷한 형식으로 예시를 작성하면 혼돈이 되므로, 형식은 다르게 입력하는 것을 권장한다.
 
 2. 예시에서
	- 1. 어떻게 2. 저렿게 3. 요렇게
	- 이런 방식으로 명확하게 예시를 작성해주면 좀 더 명확성이 올라가게 된다.
	- 따라서, 긴 문장으로 명령하는 것은 판단하기 어렵다.

3. 파일을 읽어오게 하는 방식도 가능한가?
	- 파일을 갖고오는 권한이 프롬프트에는 없다.
	- 특정 함수를 통해서 어떤 파일을 실행할 수 있도록 하는 방법은 있다.
		- part2, 3에서 도구라는 개념에서 배울 예정
		- https://platform.claude.com/docs/ko/agents-and-tools/tool-use/overview

---
## 1. **명확성(Clarity)**

- **프롬프트의 명확성**은 AI 모델과의 효과적인 소통을 위한 핵심 요소

- 불필요한 내용을 제외하고 **핵심 요구사항**에만 집중하여 작성

- 원하는 결과물에 대해 **구체적이고 정확한 지시**를 제공

In [ ]:
from langchain_core.prompts import PromptTemplate

# 명확한 지시사항이 포함된 프롬프트 템플릿
clear_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""
    주제: {topic}
    
    다음 기준을 반드시 준수하여 설명하시오:
    1. 정확히 3문장으로 작성할 것 (Bullet point 사용하여 구분)
    2. 각 문장은 20단어 이내로 작성할 것
    3. 전문 용어는 괄호 안에 간단한 설명을 포함할 것
    """
)

# LCEL 체인
clear_chain = clear_prompt | llm 

# 테스트
result = clear_chain.invoke({"topic": "인공지능"})
pprint(result.content)

---
## 2. **맥락성(Context)**

- **맥락 제공**은 AI가 작업의 배경과 목적을 이해하는데 필수적인 요소

- 프롬프트에 **배경 정보**, **목적**, **대상 환경**을 명확히 포함

- 적절한 맥락 제공은 AI의 **출력 품질**과 **정확도**를 크게 향상

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 맥락이 부족한 프롬프트의 예
bad_prompt = ChatPromptTemplate.from_template("""
당신은 친절한 AI 어시스턴트입니다. 사용자의 질문에 답변해주세요.
사용자 질문: {user_question}                                              
""")

# 맥락이 풍부한 프롬프트의 예
good_prompt = ChatPromptTemplate.from_template("""
사용자 질문: {user_question}

배경: 65세 이상 노인을 대상으로 하는 스마트폰 교육 프로그램을 진행하고 있습니다.
목적: 처음 스마트폰을 사용하는 노인들이 기본 기능을 쉽게 익힐 수 있도록 돕고자 합니다.
대상: 디지털 기기 사용 경험이 거의 없는 노인입니다.

위 맥락을 고려하여 응답해주세요.

응답 형식:
- 쉬운 용어 사용
- 단계별 설명
- 구체적인 예시 포함
""")


# LCEL 체인
good_chain = good_prompt | llm 
bad_chain = bad_prompt | llm

# 테스트
question = "아이폰에 카카오톡 설치하는 방법을 알려주세요."
good_result = good_chain.invoke({"user_question": question})
bad_result = bad_chain.invoke({"user_question": question})

print("맥락이 부족한 프롬프트의 결과")
pprint(bad_result.content)
print("-"*100)

print("맥락이 풍부한 프롬프트의 결과")
pprint(good_result.content)

---
## 3. **구조화(Structure)**

- 프롬프트 입력의 구조화: PromptTemplate, ChatPromptTemplate 사용
- LLM 출력의 구조화: OutputParser, Schema 사용

- 기대효과:
    - 일관된 형식의 입출력 보장
    - 데이터 처리 및 후속 작업의 용이성
    - 오류 처리의 체계화
    - 재사용성 향상

### 1) 프롬프트 템플릿 (Prompt Template)

- **프롬프트 템플릿**은 LLM과의 상호작용을 구조화하는 핵심 도구

- 템플릿은 다양한 입력값으로 **재사용**이 가능하며 **유연한 변수 처리**를 지원

- **검증 기능**과 **부분 포맷팅**을 통해 프롬프트 작성의 안정성을 보장

`(1) 기본 템플릿`



In [ ]:
from langchain_core.prompts import PromptTemplate

# 템플릿 정의
template = """
다음 주제에 대해 설명해주세요: {topic}
포함해야 할 내용: {content}
글자수: {length}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["topic", "content", "length"]
)

# 템플릿 사용
formatted_prompt = prompt.format(
    topic="인공지능",
    content="정의, 역사, 응용분야",
    length="500자"
)

print(formatted_prompt)

`(2) from_template 메소드`

In [ ]:
from langchain_core.prompts import PromptTemplate

# 템플릿 문자열 정의
template = """
다음 주제에 대해 설명해주세요: {topic}
포함해야 할 내용: {content}
글자수: {length}
"""

# 템플릿 생성 (템플릿 문자열 지정)
prompt = PromptTemplate.from_template(template)

# 템플릿 사용 (입력 변수 지정)
formatted_prompt = prompt.format(
    topic="인공지능",
    content="정의, 역사, 응용분야",
    length="500자"
)

print(formatted_prompt)

`(3) 템플릿 검증 및 부분 포맷팅`

In [ ]:
from langchain_core.prompts import PromptTemplate

# 템플릿 생성 
template = PromptTemplate(
    template="{product}의 다음 특징을 분석해주세요: {feature}",  # 템플릿 문자열
    input_variables=["product", "feature"],  # 입력 변수 목록
    validate_template=True  # 템플릿 유효성 검증
)

# 부분적으로 변수 채우기
partial_prompt = template.partial(product="스마트폰")

print(f"부분적으로 변수 채워진 템플릿: {partial_prompt}")
print("-"*100)

# 나중에 나머지 변수 채우기
final_prompt1 = partial_prompt.format(feature="카메라")

print(f"나머지 변수 채워진 템플릿: {final_prompt1}")
print("-"*100)

# 다른 특징을 분석하도록 변수 변경
final_prompt2 = partial_prompt.format(feature="배터리 수명")

print(f"다른 변수 채워진 템플릿: {final_prompt2}")
print("-"*100)

# 모든 변수를 한 번에 채우기
final_prompt3 = template.format(product="노트북", feature="배터리 수명")

print(f"모든 변수 채워진 템플릿: {final_prompt3}")

In [ ]:
# 템플릿 유효성 검증 실패
try:
    invalid_prompt = template.format(product="스마트폰")
except ValueError as e:
    print(f"템플릿 유효성 검증 실패: {e}")

### 2) 챗 프롬프트 템플릿 (Chat Prompt Template)

- **챗 프롬프트 템플릿**은 대화형 AI와의 상호작용을 위한 특화된 템플릿

- 시스템/사용자/어시스턴트 등 **다양한 역할**의 메시지를 구조화

- 대화의 **맥락과 흐름**을 유지하면서 일관된 상호작용 가능

`(1) 메시지 템플릿 사용`

In [ ]:
### 복잡한 템플릿 구성

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 개별 메시지 템플릿 생성
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {role} 전문가입니다. {style} 스타일로 답변해주세요."
)

human_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)

# from_messages 메소드 사용 (여러 개의 메시지들을 원소로 갖는 리스트로 구성)
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    human_message
])

# 템플릿 사용
formatted_prompt = chat_prompt.format(
    role="인공지능",
    style="친절한",
    question="인공지능의 정의를 설명해주세요."
)

print(formatted_prompt)

`(2) 문자열 템플릿 사용`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지와 사용자 메시지를 포함한 템플릿 정의
template = """
당신은 {role} 전문가입니다. {style} 스타일로 답변해주세요.

{question}
"""

# from_template 메소드 사용 (단일 템플릿 문자열 직접 사용)
chat_prompt = ChatPromptTemplate.from_template(template)

# 템플릿 사용
formatted_prompt = chat_prompt.format(
    role="인공지능",
    style="친절한",
    question="인공지능의 정의를 설명해주세요."
)

# ChatPromptTemplate을 직접 생성 - 이때는 HumanMessage로 처리됨 (SystemMessage는 별도로 추가해야 함)
print(formatted_prompt)

### 2)  OutputParser 활용

- **OutputParser**는 LLM의 출력을 다양한 데이터 형식으로 변환하는 도구

- **문자열**, **JSON**, **XML** 등 여러 형식의 파싱을 지원

- 파싱된 출력은 **다른 시스템**이나 **프로세스**와 연동하는데 유용

`(1) JSONOutputParser`
- LLM의 출력을 **구조화된 JSON**으로 변환
- 파서는 출력의 **데이터 유효성**을 검증하고 일관된 형식을 보장

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

# 관광지 정보를 위한 Pydantic 모델 정의 (이름, 위치, 카테고리, 주요 관람 포인트)
class TouristSpot(BaseModel):
    name: str = Field(description="관광명소 이름")
    location: str = Field(description="위치 (구/동 정보)")
    category: str = Field(description="카테고리 (궁궐/박물관/쇼핑 등)")
    highlights: List[str] = Field(description="주요 관람 포인트")

# JsonOutputParser 파서 설정 (Pydantic 모델 지정)
parser = JsonOutputParser(pydantic_object=TouristSpot)

# parser의 get_format_instructions 메소드 사용 (포맷 지시사항 출력)
pprint(parser.get_format_instructions())

In [ ]:
# 프롬프트 템플릿 정의 (parser의 get_format_instructions 메소드 사용)
prompt = PromptTemplate(
    template="""서울의 다음 관광명소에 대한 상세 정보를 제공해주세요.
{format_instructions}

관광지: {spot_name}
""",
    input_variables=["spot_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 완성된 프롬프트 템플릿 출력 (포맷 지시사항 포함)
pprint(prompt.format(spot_name="경복궁"))

In [ ]:
# 체인 구성
chain = prompt | llm | parser

# 실행 예시
result = chain.invoke({
    "spot_name": "경복궁"
})

# 결과 출력
pprint(result)

`(2) XMLOutputParser`
- LLM의 출력을 **구조화된 XML** 형식으로 변환
- XML 구조는 **계층적 데이터**를 표현하는데 효과적
- 패키지 설치: defusedxml 

In [ ]:
from langchain_core.output_parsers import XMLOutputParser

# XML 파서 설정 - 원하는 태그 구조 정의
parser = XMLOutputParser(tags=["tourist_spot", "name", "location", "category", "highlights", "point"])

# parser의 get_format_instructions 메소드 사용 (포맷 지시사항 출력)
pprint(parser.get_format_instructions())

In [ ]:
# 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="""서울의 다음 관광명소에 대한 상세 정보를 XML 형식으로 제공해주세요.
{format_instructions}

관광지: {spot_name}""",
    input_variables=["spot_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()}  
)

# 완성된 프롬프트 템플릿 출력 (포맷 지시사항 포함)
pprint(prompt.format(spot_name="경복궁"))

In [ ]:
# 체인 구성
chain = prompt | llm | parser

# 실행 예시
result = chain.invoke({
    "spot_name": "경복궁"
})

# 결과는 Python 딕셔너리 형태로 파싱되어 반환됨 
pprint(result)

In [ ]:
# XML 형식의 결과를 다시 파싱하여 출력 (가장 상위 태그를 제외한 내용만 출력)
result['tourist_spot']

`(3) 사용자 정의(Custom) OutputParser`
- **사용자 정의 파서**는 **RunnableLambda**나 **RunnableGenerator**를 활용하여 구현
- 특정 비즈니스 요구사항에 맞는 **맞춤형 출력 형식**을 정의할 수 있음 
- 복잡한 **데이터 변환**과 **후처리 로직**을 유연하게 구현할 수 있음 

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import PromptTemplate
from typing import Dict

# 사용자 정의 파서
def parse_tourist_spot(ai_message: AIMessage) -> Dict:
    """관광지 정보를 딕셔너리로 파싱"""
    lines = ai_message.content.split('\n')
    return {
        "name": lines[0] if lines else "",
        "description": ' '.join(lines[1:]) if len(lines) > 1 else ""
    }

# 프롬프트 템플릿
prompt = PromptTemplate(
    template="다음 관광지에 대해 첫 줄에 이름, 다음 줄부터 설명을 100자 내외로 작성해주세요:\n{spot_name}",
    input_variables=["spot_name"]
)

# 체인 구성 (invoke 방식)
invoke_chain = prompt | llm | parse_tourist_spot

# 체인 실행
result = invoke_chain.invoke({
    "spot_name": "경복궁"
})

pprint(result)

In [ ]:
## 위 구현을 stream 방식으로 변경

from langchain_core.runnables import RunnableGenerator
from langchain_core.messages import AIMessageChunk
from typing import Iterable
import time

# 사용자 정의 파서 (stream 방식) 
def streaming_parse_tourist_spot(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    """실시간으로 관광지 정보 파싱"""
    for chunk in chunks:
        yield f"🏛 {chunk.content}"   # 🏛 이모지 추가 

# stream 방식 체인 구성 
streaming_chain = prompt | llm | RunnableGenerator(streaming_parse_tourist_spot)

# 체인 실행 
for chunk in streaming_chain.stream({"spot_name": "광화문"}):
    print(chunk)
    time.sleep(0.1)   # 시간 지연 추가 

**[참고] 이모지(emoji) 입력 방법**

1. Windows에서:
    - `Windows 키 + .` (윈도우 키와 마침표를 동시에 누름)
    - 또는 `Windows 키 + ;` (윈도우 키와 세미콜론을 동시에 누름)

2. Mac에서:
    - `fn(지구본) 키 + E` 

`(3) 구조화된 출력 프롬프트 (Structured Output Prompts)`
   - 일관된 형식의 응답
   - 데이터 처리 용이

In [ ]:
# typing 사용
from typing import TypedDict, Annotated 

class AnalysisResult(TypedDict):
    """분석 결과 스키마"""
    summary: Annotated[str, ..., "핵심 요약"]
    keywords: Annotated[list[str], ..., "주요 키워드"]
    sentiment: Annotated[str, ..., "긍정/부정/중립"]

structured_llm = llm.with_structured_output(AnalysisResult)

# LCEL
chain = step_prompt | structured_llm

# 질문
text = """
양자 컴퓨팅은 양자역학의 원리를 바탕으로 데이터를 처리하는 새로운 형태의 계산 방식이다.
기존의 고전적 컴퓨터는 0과 1로 이루어진 이진법(bit)을 사용하여 데이터를 처리하지만,
양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용하여 훨씬 더 복잡하고 빠른 계산을 수행할 수 있다.

큐비트는 동시에 0과 1의 상태를 가질 수 있는 양자 중첩(superposition) 상태를 활용하며,
이를 통해 병렬 계산과 같은 고급 기능이 가능하다.
"""

output = chain.invoke({"text": text})
pprint(output)

In [ ]:
# pydantic 사용
from typing import List, Literal
from pydantic import BaseModel, Field

class AnalysisResult(BaseModel):
    """텍스트 분석 결과를 담는 스키마"""
    
    summary: str = Field(
        ...,  # ... 은 required 필드를 의미 (필수 입력, None 허용 안함)
        description="텍스트의 핵심 내용 요약"
    )
    
    keywords: List[str] = Field(
        ...,
        description="텍스트에서 추출한 주요 키워드"
    )
    
    sentiment: Literal["긍정", "부정", "중립"] = Field(
        ...,
        description="텍스트의 전반적인 감정 분석 결과"
    )

structured_llm = llm.with_structured_output(AnalysisResult)

# LCEL
chain = step_prompt | structured_llm

# 질문
output = chain.invoke({"text": text})
print(output)
print(type(output))
print("-"*100)
print(output.summary)
print(output.keywords)
print(output.sentiment)

---

# [실습 프로젝트]

### 맞춤형 학습 도우미 챗봇 만들기

- 특정 주제에 대한 학습을 돕는 챗봇을 만들어봅니다. 앞서 배운 프롬프트 유형들을 조합하여 활용합니다.

- (1) 퀴즈 문제 예시를 보고, (2) 개념 설명 체인을 완성합니다. 

`(1) 퀴즈 문제 출제 (예시)`

In [3]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 퀴즈 문제 스키마
class QuizQuestion(BaseModel):
    """퀴즈 문제 스키마"""
    question: str = Field(..., description="퀴즈 문제")
    options: List[str] = Field(..., description="보기 (4개)")
    correct_answer: int = Field(..., description="정답 번호 (1-4)")
    explanation: str = Field(..., description="정답 설명")


# 퀴즈 생성을 위한 구조화된 출력 프롬프트
# 주제, 난이도(상중하) 아래에 참고사항을 넣어 환각을 줄일 수 있도록 해본다.
quiz_prompt = PromptTemplate(
    template="""다음 주제에 대한 퀴즈 문제를 만들어주세요:
    
주제: {topic}
난이도(상/중/하): {difficulty}

다음 조건을 만족하는 퀴즈를 생성해주세요:
1. 문제는 명확하고 이해하기 쉽게
2. 4개의 보기 제공
3. 정답과 오답은 비슷한 수준으로
4. 상세한 정답 설명 포함""",
    input_variables=["topic", "difficulty"]
)

# LLM 정의
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

# 구조화된 출력 파서 설정
structured_llm = llm.with_structured_output(QuizQuestion)

# LCEL 체인 구성
chain = quiz_prompt | structured_llm

# 질문 실행
output = chain.invoke({"topic": "인공지능", "difficulty": "상"})

# 결과 출력
pprint(f"퀴즈 문제: {output.question}")
pprint(f"보기: {output.options}")
pprint(f"정답: {output.correct_answer}")
pprint(f"정답 설명: {output.explanation}")

"퀴즈 문제: 딥러닝에서 '과적합(overfitting)' 문제를 해결하기 위한 대표적인 방법 중 하나가 아닌 것은 무엇인가?"
("보기: ['드롭아웃(Dropout) 기법 적용', '데이터 증강(Data Augmentation)', '모델의 복잡도 증가', '조기 "
 "종료(Early Stopping)']")
'정답: 3'
('정답 설명: 과적합은 모델이 학습 데이터에 너무 치중하여 일반화 성능이 떨어지는 현상입니다. 이를 방지하기 위해 드롭아웃, 데이터 증강, '
 '조기 종료 등 다양한 방법을 사용합니다. 하지만 모델의 복잡도를 증가시키면 오히려 과적합이 심해질 수 있으므로 이는 과적합 해결 방법이 '
 '아닙니다.')


/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=QuizQuestion(question="...법이 아닙니다.'), input_type=QuizQuestion])
  return self.__pydantic_serializer__.to_python(


`(2) 개념 설명 체인 (문제)`

In [ ]:
# 개념 설명을 위한 스키마
class ConceptExplanation(BaseModel):
    """개념 설명 스키마"""
    topic: None
    explanation: None
    examples: None
    related_concepts: None

# 개념 설명을 위한 지시형 프롬프트
concept_prompt = PromptTemplate(
    template=None,
    input_variables=["topic", "difficulty"]
)

# 구조화된 출력 파서 설정
structured_llm = None

# LCEL 체인 구성
chain = None

# 질문 실행
# output = chain.invoke({"topic": "인공지능", "difficulty": "하"})

# 강사님 요청사항 : 질문 실행 - level 넣어서, template이 None이 아닌 level별로 출력될 수 있도록 처리
# output = chain.invoke({"topic": "인공지능", "difficulty": "하", "level": ""})

# 결과 출력
pprint(f"주제: {output.topic}")
pprint(f"설명: {output.explanation}")
pprint(f"예시: {output.examples}")
pprint(f"관련 개념: {output.related_concepts}")

### [실습프로젝트 강사님 코드]

In [ ]:
# 개념 설명을 위한 스키마
class ConceptExplanation(BaseModel):
    """개념 설명 스키마"""
    topic: str = Field(..., description="설명할 주제")
    explanation: str = Field(..., description="주제에 대한 상세 설명")
    examples: list[str] = Field(..., description="이해를 돕는 구체적인 예시 3개")
    related_concepts: list[str] = Field(..., description="관련된 개념 3개")

# 개념 설명을 위한 지시형 프롬프트
concept_prompt = PromptTemplate(
    template="""다음 주제에 대해 상세히 설명해주세요:

주제: {topic}
난이도(상/중/하): {difficulty}

다음 조건을 만족하는 설명을 생성해주세요:
1. 난이도에 맞는 쉬운 용어 사용
2. 구체적인 예시 3개 포함
3. 관련 개념 3개 제시
4. 체계적이고 이해하기 쉬운 설명""",
    input_variables=["topic", "difficulty"]
)

# 구조화된 출력 파서 설정
structured_llm = llm.with_structured_output(ConceptExplanation)

# LCEL 체인 구성
chain = concept_prompt | structured_llm

# 질문 실행
output = chain.invoke({"topic": "인공지능", "difficulty": "하"})

# 결과 출력
pprint(f"주제: {output.topic}")
pprint(f"설명: {output.explanation}")
pprint(f"예시: {output.examples}")
pprint(f"관련 개념: {output.related_concepts}")